# Libraries, organizing your code

The classic HDL way is to write a simple file with one `@block` decorated hardware unit and import it as a module. However this is not sufficient in some cases when full portability or extended configureability is desired.

Also, one might use a library of existing, external VHDL or Verilog code.

## User libraries

When starting a new set of hardware elements from scratch, the best approach is to create `@cyrite_factory.block_component` objects inside a `@cyrite_factory.Module` class descendant. By well- derival, hardware modules can be reused.

In particular, when a hardware description should simulate and synthesize, it is desirable to have a base class and use inheritance to implement more
features or architecture-specific variants.

When designing internal libraries that mostly create procedural code in IRL notation, you might however use bare metal LibraryModule classes.

**Note**: LibraryModule `@block_components` can refer to external objects (@block or @blackbox components), but will not emit them during elaboration.

In [1]:
import myirl

class MyLib(myirl.LibraryModule):
    t_mode = myirl.enum('ADD', 'SUB')
    @myirl.block_component
    def addsub(self,
              a : myirl.Signal,
              b : myirl.Signal,
              m : (type(t_mode.ADD), myirl.Signal.Type(t_mode.ADD)),
              q : myirl.Signal.Output):

        @myirl.genprocess_ctx(a, b)
        def worker(ctx):
            yield [
                ctx.If(m == self.t_mode.ADD).Then(
                    q .set (a + b)
                ).Else(
                    q .set (a - b)
                )
            ]

        return instances()

Note that the `addsub` interface is *not* strict, as it allows passing a constant instead of a signal. This will possibly generate several implementations, depending on the argument, as we can see below.

### Including libraries in designs

To make use of automatic collection of relevant library files, instance a library in the class body as below:

In [2]:
from cyhdl import *

In [3]:
class MyDesign(cyrite_factory.Module):

    mylib = MyLib("mine")

    @cyrite_factory.block_component
    def my_unit(self, clk : ClkSignal,
                a : Signal.Type(intbv, 8),
                b : Signal.Type(intbv, 9).Output):

        y = b.clone()
        z = b.clone()

        uut0 = self.mylib.addsub(a, a,
                                self.mylib.t_mode.ADD,
                               y)

        uut1 = self.mylib.addsub(a, a,
                                self.mylib.t_mode.SUB,
                               z)

        wires = [
            b    .wireup (y ^ z)
        ]
        
        return instances()


Note the `addsub` unit is instanced twice with different parameters and we are being warned about a constant 'pass through' argument. We are creating an instance of this design:

In [4]:
from cyrite.simulation import ghdl

d = MyDesign("test", ghdl.GHDL)
a = Signal(intbv()[8:])
q = Signal(intbv()[9:])
clk = ClkSignal()

uut = d.my_unit(clk, a, q)
files = d.elab(targets.VHDL)
files

 Declare obj 'my_unit' in context '(MyDesign 'test')'(<class '__main__.MyDesign'>) 
 Declare obj 'addsub' in context '(LIB: MyLib 'mine')'(<class '__main__.MyLib'>) 
 Module mine: Existing instance addsub, rename to addsub_1 
 Skip registration of (LIB: MyLib 'mine')/<class '__main__.MyLib'> 
 Writing 'my_unit' to file /tmp/myirl_test__9cw1wdg/my_unit.vhdl 
 Creating library file module_defs.vhdl 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:116: UserWarning: Fallback: Pass through other argument for arg m (<class 'myirl.enumtypes.EnumItem'>)
  base.warnings.warn(msg)


['/tmp/myirl_test__9cw1wdg/my_unit.vhdl', 'module_defs.vhdl']

This does not yet include the library files. We collect them manually from the instanced library:

In [5]:
d.mylib.collect_libfiles(targets.VHDL)

 Writing 'addsub_1' to file ./addsub_1.vhdl 
 Writing 'addsub' to file ./addsub.vhdl 
 Not emitting design types library 


['./addsub_1.vhdl', './addsub.vhdl']

We can see in the result that two implementations are generated for each constant value that is passed.

In [6]:
d.mylib.components

['addsub_obj_MyLibu_8u_8E_ADD1u_9', 'addsub_obj_MyLibu_8u_8E_SUB1u_9']

We display the actual unit instantiation in HDL:

In [7]:
!cat {files[0]}

-- File generated from source:
--     None
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity my_unit is
    port (
        clk : in std_ulogic;
        a : in unsigned(7 downto 0);
        b : out unsigned(8 downto 0)
    );
end entity my_unit;

architecture myIRL of my_unit is
    -- Local type declarations
    -- Signal declarations
    signal y : unsigned(8 downto 0);
    signal z : unsigned(8 downto 0);
begin
    
    -- Instance addsub
    inst_addsub_0: entity work.addsub
    port map (
        a => a,
        b => a,
        q => y
    );
    
    -- Instance addsub_1
    inst_addsub_1: entity work.addsub_1
    port map (
        a => a,
        b => a,
        q => z
    );
    b <= (y xor z);
end architecture myIRL;



### Issues

Putting the library in the `Module` header has one drawback: since this is a persistent instance inside the class definition, it will also record all implementations of its `@block_components`. Thus, several calls, for instance within a complex pytest regression test may emit the same module several times. This can cause errors with direct synthesis, therefore you might want to put an explicit `self.mylib.clear()` command in the `Module.__init__` function when creating different configurations.

### Conclusion and guidelines

`LibraryModule` classes allow block components with non-strict interfaces, therefore they are considered ad-hoc libraries such as a 'work' library in VHDL. They can be created in inhomogenous variants, depending on parameters. Also, only implementations that are used will be created within a project. So they are not suitable to create VHDL library packages.

## External HDL libraries

External HDL files are technically treated as black boxes and are referenced as a stub within the target design.

Some targets might support file-only blackboxes. The `@blackbox` decorator itself just reserves the stub, but does not provide means to pull the file.

For a library, this is inconvenient, we'd rather see that automated and explicitely mapping to a target language.
The `@blackbox_verilog` decorator takes a `path_prefix` to the file whose filename is automatically constructed from the function name. Likewise, there is a `@blackbox_vhdl` decorator.

In [8]:
from cyhdl import *

Bool = Signal.Type(bool)
Byte = Signal.Type(intbv()[8:])

from myirl.blackbox_ext import *

@blackbox_verilog(path_prefix="./verilog")
def debug(clk : ClkSignal, en : Bool, i_data : Byte, o_data : Byte):
    pass

Verify the reference to the external Verilog file:

In [9]:
debug.get_sources()

['./verilog/debug.v']

### Existing VHDL packages

If you wish to integrate an existing VHDL library package, you can derive from a blackbox `ExternalLibrary` class and define `@blackbox_component` methods for each component declaration. This is not covered by this documentation in detail.

For a VHDL library, instancing a library method will:

* Create a `use.<library>.all` directive in the header
* Reference an instance as *builtin*, i.e. `inst : module`, unlike a reference to the current work library: `inst : entity work.module`.

For Verilog, this will just add a library file to the project.



## Builtin Primitives

**Warning** Synthesizer specific

Some synthesis packages offer a library of built-in primitives that can be instanced directly from your code as a component.

Even though they can be considered white or gray boxes, they are treated as a special `@blackbox` component on the IRL kernel level, because they may not provide a native Python description that may possibly simulate. In most cases, external vendor specific simulation models are provided.

For **yosys** and a few supported architectures, wrappers are provided for the openly available simulation models.


### Example

In [10]:
from yosys.builtins import builtins

@block
def unit(clk : ClkSignal, data_in : Signal, data_out : Signal.Output):
    
    # tmp = data_out.clone()
    
    inst_dff = builtins.Dff(CLK = clk,
                            D = data_in,
                            Q = data_out,
                            CLK_POLARITY = True,
                            WIDTH = data_in.size()
                           )
    
    return [ inst_dff ]

In [11]:
a, b = [ Signal(intbv()[12:], name = n) for n in "ab" ]
clk = ClkSignal()

u = unit(clk, a, b)

from myirl.targets.pyosys import RTLIL
RTL = RTLIL("default")

objs = u.elab(RTL)
d = objs[0]

DEBUG INIT IN CONTEXT Dff (LIB: _Builtins '_yosys_builtins_')
 [_builtin_method 'Dff/Dff'] blackbox not returning instances 
 DEBUG components ['unitu_1u_12u_12'] (DesignModule 'unit') 
 Adding module with name `unit` 
 PARAM CLK_POLARITY --> True 
 PARAM WIDTH --> 12 
 FINALIZE implementation `unit` of `unit` 


/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:203: UserWarning: @component `unit` interface :Output data_out not driven
  base.warnings.warn(self._err("Output %s not driven" % \


In [12]:
d.display_rtl(selection = unit, fmt='dot')

In [13]:
from yosys import display
display.display_dot(d.name)

## General vendor blackbox issues

If a FPGA vendor's blackbox is to be instanced directly, it may be important to sort out at an early stage if:

* It is supplied with a binding for synthesis
* If is supplied with a non-encrypted simulation file for the OpenSource simulators GHDL or Verilog

In the case of encrypted simulation files, the only legal way to output towards simulation is:

* Choose the provided simulator for your FPGA target
* Output your design to Verilog

### CXXRTL compatibility

From the vendor's perspective, the it is to be kept in mind that black box models must be synthesizeable using yosys and may not contain specific timing information. Therefore, all sorts of clock generators such as PLLs can not be compiled into a simulation.

## Inline blackboxes

Inline blackboxes are called like functions and create an expression. Silently, an instance of a builtin-primitive and an output signal is created. From the front, an inline component is considered a blackbox with a referred implementation, the latter being a usual block component with a possibly strict interface.

Inline blackboxes *generate* their interface ad-hoc, thus have freedom to proceduralle create instances.

Currently, inline blackboxes are not documented in detail for *cyhdl* usage.